# Installation des différents package

In [1]:
!pip install semantic-kernel python-dotenv google-cloud-bigquery pyyaml google-cloud-bigquery-storage

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.8/894.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import des variables d'environnement

N'oubliez pas de copier-coller le fichier .env à la racine du code

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [5]:
GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="colas-training-service-account.json"

# Import des librairies nécessaires

In [9]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
import logging
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
import yaml
import os
from semantic_kernel.functions import kernel_function
from datetime import datetime
import requests
from typing import Annotated, Optional, List, Dict, Any
from google.cloud import bigquery
import json

# Définition du Tool permettant de récupérer la météo

In [7]:
class WeatherPlugin:
    """Weather plugin for Semantic Kernel with Open-Meteo API integration."""

    @kernel_function(
    name="get_current_weather",
    description="Fetches current weather for given coordinates using Open-Meteo API"
    )
    def get_current_weather(
        self,
        latitude: float,
        longitude: float
    ) -> dict:
        """Fetches current weather for given coordinates.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate

        Returns:
            dict: Weather details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": [
                "temperature_2m",
                "relative_humidity_2m",
                "precipitation",
                "weather_code",
                "wind_speed_10m",
            ],
            "timezone": "auto",
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            current = data["current"]

            weather_description = self._get_weather_description(current["weather_code"])

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "description": weather_description,
                "temperature": current["temperature_2m"],
                "humidity": current["relative_humidity_2m"],
                "wind_speed": current["wind_speed_10m"],
                "precipitation_1h": current["precipitation"],
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_weather_forecast",
        description="Fetches daily weather forecast for up to 16 days using Open-Meteo API"
    )
    def get_weather_forecast(
        self,
        latitude: float,
        longitude: float,
        days: int = 7
    ) -> dict:
        """Fetches daily weather forecast.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate
            days: Number of days for forecast (default 7, max 16)

        Returns:
            dict: Forecast details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        days = min(days, 16)

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": [
                "temperature_2m_max",
                "temperature_2m_min",
                "weather_code",
                "precipitation_sum",
                "wind_speed_10m_max",
                "wind_gusts_10m_max",
                "wind_direction_10m_dominant",
            ],
            "timezone": "auto",
            "forecast_days": days,
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            daily = data["daily"]

            forecasts = []
            for i in range(len(daily["time"])):
                weather_description = self._get_weather_description(daily["weather_code"][i])
                forecasts.append(
                    {
                        "date": daily["time"][i],
                        "temp_max": daily["temperature_2m_max"][i],
                        "temp_min": daily["temperature_2m_min"][i],
                        "description": weather_description,
                        "precipitation": daily["precipitation_sum"][i],
                        "wind_speed_max": daily["wind_speed_10m_max"][i],
                        "wind_gusts_max": daily["wind_gusts_10m_max"][i],
                        "wind_direction": daily["wind_direction_10m_dominant"][i],
                    }
                )

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "forecasts": forecasts,
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_current_date",
        description="Fetches the current date and time"
    )
    def get_current_date(self) -> dict:
        """Fetches the current date and time.

        Returns:
            dict: Current date and time with status
        """
        try:
            current_date_time = datetime.utcnow().isoformat()
            return {"status": "success", "current_date_time": current_date_time}
        except Exception as e:
            return {"status": "error", "error_message": str(e)}

    def _get_weather_description(self, code: int) -> str:
        """Maps WMO weather code to description.

        Args:
            code: WMO weather code

        Returns:
            str: Weather description
        """
        weather_codes = {
            0: "clear sky",
            1: "mainly clear",
            2: "partly cloudy",
            3: "overcast",
            45: "fog",
            48: "depositing rime fog",
            51: "light drizzle",
            53: "moderate drizzle",
            55: "dense drizzle",
            56: "light freezing drizzle",
            57: "dense freezing drizzle",
            61: "slight rain",
            63: "moderate rain",
            65: "heavy rain",
            66: "light freezing rain",
            67: "heavy freezing rain",
            71: "slight snow",
            73: "moderate snow",
            75: "heavy snow",
            77: "snow grains",
            80: "slight rain showers",
            81: "moderate rain showers",
            82: "violent rain showers",
            85: "slight snow showers",
            86: "heavy snow showers",
            95: "thunderstorm",
            96: "thunderstorm with slight hail",
            99: "thunderstorm with heavy hail",
        }
        return weather_codes.get(code, "unknown")


# Tool Bigquery

In [8]:
class BigQueryPlugin:
    """
    A Semantic Kernel plugin for intelligent BigQuery table access.
    Automatically determines which queries to create based on user requests.
    """
    
    def __init__(self, project_id: str, dataset_id: Optional[str] = None):
        """
        Initialize the BigQuery plugin.
        
        Args:
            project_id: Google Cloud project ID
            dataset_id: Default dataset ID (optional)
        """
        self.client = bigquery.Client(project=project_id)
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.schema_cache = {}
        
    def _get_table_schema_yaml(self, table_id: str) -> str:
        """
        Get table schema in YAML format for LLM consumption.
        Uses YAML for token efficiency as recommended by Semantic Kernel best practices.
        """
        if table_id in self.schema_cache:
            return self.schema_cache[table_id]
        
        try:
            table_ref = f"{self.project_id}.{table_id}"
            table = self.client.get_table(table_ref)
            
            schema_dict = {
                'platform': 'BigQuery',
                'table': table_id,
                'description': table.description or f'Table {table_id}',
                'columns': []
            }
            
            for field in table.schema:
                column_info = {
                    'name': field.name,
                    'type': field.field_type,
                }
                if field.description:
                    column_info['description'] = field.description
                schema_dict['columns'].append(column_info)
            
            yaml_schema = yaml.dump(schema_dict, default_flow_style=False)
            self.schema_cache[table_id] = yaml_schema
            return yaml_schema
            
        except Exception as e:
            return f"Error retrieving schema: {str(e)}"
    
    @kernel_function(
        name="get_table_schema",
        description="Gets the schema of a BigQuery table including column names, types, and descriptions. Use this to understand table structure before querying."
    )
    async def get_table_schema(
        self,
        table_id: Annotated[str, "The table ID in format 'dataset.table' or 'project.dataset.table'"]
    ) -> str:
        """Retrieve table schema in YAML format."""
        return self._get_table_schema_yaml(table_id)
    
    @kernel_function(
        name="list_tables",
        description="Lists all tables in a dataset. Use this when you need to discover available tables or when the user asks about what data is available."
    )
    async def list_tables(
        self,
        dataset_id: Annotated[Optional[str], "The dataset ID. If not provided, uses the default dataset"] = None
    ) -> str:
        """List all tables in the specified dataset."""
        target_dataset = dataset_id or self.dataset_id
        
        if not target_dataset:
            return "Error: No dataset specified and no default dataset configured"
        
        try:
            dataset_ref = f"{self.project_id}.{target_dataset}"
            tables = self.client.list_tables(dataset_ref)
            
            table_list = []
            for table in tables:
                table_info = {
                    'table_id': table.table_id,
                    'full_table_id': f"{target_dataset}.{table.table_id}",
                    'table_type': table.table_type
                }
                table_list.append(table_info)
            
            return json.dumps(table_list, indent=2)
            
        except Exception as e:
            return f"Error listing tables: {str(e)}"
    
    @kernel_function(
        name="execute_query",
        description="Executes a SQL query against BigQuery. The query should be a valid BigQuery SQL SELECT statement. Use this only after understanding the table schema. Always use fully qualified table names (dataset.table)."
    )
    async def execute_query(
        self,
        query: Annotated[str, "The SQL SELECT query to execute. Must be a valid BigQuery SQL statement with fully qualified table names."],
        max_results: Annotated[int, "Maximum number of results to return. Default is 100."] = 100
    ) -> str:
        """
        Execute a BigQuery SQL query and return results.
        Implements read-only access for security.
        """
        # Security: Ensure query is SELECT only
        query_upper = query.strip().upper()
        if not query_upper.startswith('SELECT'):
            return "Error: Only SELECT queries are allowed for security reasons"
        
        # Prevent potentially dangerous operations
        dangerous_keywords = ['DROP', 'DELETE', 'INSERT', 'UPDATE', 'CREATE', 'ALTER', 'TRUNCATE']
        if any(keyword in query_upper for keyword in dangerous_keywords):
            return f"Error: Query contains prohibited keywords. Only SELECT queries are allowed."
        
        try:
            query_job = self.client.query(query)
            results = query_job.result(max_results=max_results)
            
            # Convert to list of dictionaries
            rows = []
            for row in results:
                rows.append(dict(row))
            
            result_data = {
                'row_count': len(rows),
                'total_rows': results.total_rows,
                'data': rows[:max_results]
            }
            
            return json.dumps(result_data, indent=2, default=str)
            
        except Exception as e:
            return f"Error executing query: {str(e)}"
    
    @kernel_function(
        name="validate_query",
        description="Validates a SQL query without executing it. Use this to check if a query is syntactically correct before execution."
    )
    async def validate_query(
        self,
        query: Annotated[str, "The SQL query to validate"]
    ) -> str:
        """Validate a query using BigQuery dry run."""
        try:
            job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
            query_job = self.client.query(query, job_config=job_config)
            
            return json.dumps({
                'valid': True,
                'total_bytes_processed': query_job.total_bytes_processed,
                'message': 'Query is valid'
            }, indent=2)
            
        except Exception as e:
            return json.dumps({
                'valid': False,
                'error': str(e)
            }, indent=2)
    
    @kernel_function(
        name="get_table_preview",
        description="Gets a preview of table data (first 10 rows). Use this to understand the actual data in a table."
    )
    async def get_table_preview(
        self,
        table_id: Annotated[str, "The table ID in format 'dataset.table'"]
    ) -> str:
        """Get a preview of table data."""
        query = f"SELECT * FROM `{self.project_id}.{table_id}` LIMIT 10"
        return await self.execute_query(query, max_results=10)

In [ ]:

async def main():
    # Initialize the kernel
    kernel = Kernel()

    # Add Azure OpenAI chat completion
    chat_completion = AzureChatCompletion(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),
        api_key=os.getenv("API_KEY"),
        base_url=os.getenv("BASE_URL"),
    )
    kernel.add_service(chat_completion)

    # Set the logging level for  semantic_kernel.kernel to DEBUG.
    setup_logging()
    logging.getLogger("kernel").setLevel(logging.DEBUG)

    # Add BigQuery plugin
    bq_plugin = BigQueryPlugin(
        project_id="colas-training",
        dataset_id="colas_data"  # Optional default dataset
    )
    kernel.add_plugin(
        bq_plugin,
        plugin_name="BigQuery"
    )

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Create a history of the conversation
    history = ChatHistory()

    # Initiate a back-and-forth chat
    userInput = None
    while True:
        # Collect user input
        userInput = input("User > ")

        # Terminate the loop if the user says "exit"
        if userInput == "exit":
            break

        # Add user input to the history
        history.add_user_message(userInput)

        # Get the response from the AI
        result = await chat_completion.get_chat_message_content(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
        )

        # Print the results
        print("Assistant > " + str(result))

        # Add the message from the agent to the chat history
        history.add_message(result)

await main()